This is version 2 of my Fantasy Hockey Analyzer. The purpose of this notebook is to predict the number of fantasy points every hockey player in the league will get based on previous years' performance.

This notebook primarily uses data from moneypuck.com for analysis, and it also uses data from rotowire.com to get +/- for each player.

Section 1: Parameters and Modules

These are the variables that can be adjusted. My model is an ensemble model consisting of neural nets and random forests, with data going back one, two, and three years.

In [1]:
# Set these values to the appropriate ammonts

current_year = 2025
common_number = 100
number_of_one_year_neural_nets = common_number
number_of_two_year_neural_nets = common_number
number_of_three_year_neural_nets = common_number
number_of_one_year_random_forests = common_number
number_of_two_year_random_forests = common_number
number_of_three_year_random_forests = common_number

# This is the breakdown of how many fantasy points a player gets for each category
points_dict = {
    "Goals":5, 
    "Assists":3, 
    "+/-":1.5, 
    "PIM":-0.25, 
    "PP_Goals":4, 
    "PP_Assists":2, 
    "SH_Goals":6, 
    "SH_Assists":4, 
    "Faceoffs_Won":0.25, 
    "Faceoffs_Lost":-0.15, 
    "Hits":0.5, 
    "Blocked_Shots":0.75
    }


The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [2]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone

import my_module_v2 as mx

Section 2: Data Gathering and Cleaning

This section compiles the Moneypuck and Rotowire data into a format that is usable by the ML models.

In [ ]:
yearly_player_data = []

for year in range(2010, 2011):

    moneypuck_data = mx.get_moneypuck_data(year)
    rotowire_data = mx.get_rotowire_data(year)

    combined_data = mx.combine_dataframes(moneypuck_data, rotowire_data)
    display(moneypuck_data.head())

    display(rotowire_data.head())

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
0,8470144,2010,Frans Nielsen,NYI,C,other,71,2884.0,62.0,38.23,...,13.51,14.39,150.0,130.0,0.00,0.00,0.0,0.0,0.0,0.0
1,8470144,2010,Frans Nielsen,NYI,C,all,71,75672.0,1692.0,48.22,...,131.32,158.11,2675.0,3171.0,0.00,0.00,0.0,0.0,0.0,0.0
2,8470144,2010,Frans Nielsen,NYI,C,5on5,71,50658.0,1130.0,48.22,...,86.94,112.55,2089.0,2607.0,1.58,2.15,43.0,46.0,35.0,38.0
3,8470144,2010,Frans Nielsen,NYI,C,4on5,71,11891.0,313.0,48.22,...,3.99,22.73,58.0,338.0,0.03,0.06,2.0,4.0,2.0,2.0
4,8470144,2010,Frans Nielsen,NYI,C,5on4,71,10239.0,187.0,46.30,...,23.17,3.50,333.0,61.0,0.19,0.00,3.0,1.0,2.0,0.0


,Player Name,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,G.1,A.1,G.2,A.2,Hits,BS
0,Corey Perry,TB,RW,82,50,48,98,9,104,290,11,14,17,4,1,64,41
1,Steven Stamkos,TB,C,82,45,46,91,3,74,272,8,17,19,0,0,84,37
2,Jarome Iginla,FA,RW,82,43,43,86,0,40,289,6,14,16,0,0,103,28
3,Daniel Sedin,FA,LW,82,41,63,104,30,32,266,10,18,24,0,0,13,12
4,Ryan Kesler,FA,C,82,41,32,73,24,66,260,7,15,15,3,1,124,80
